In [20]:
# automatically load changes in Python code without restarting Kernel
%reload_ext autoreload
%autoreload 2

In [21]:
from elasticsearch import Elasticsearch
import elasticsearch_functions as ef

import warnings
warnings.filterwarnings("ignore")

In [22]:
es_host = "http://localhost:9200"

es = Elasticsearch(
    hosts=es_host
)

es.info()   # check if connection was successful

{'name': 'a92da01f1532',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': '-Q71RHpESH-y0QUvpgi14w',
 'version': {'number': '7.17.7',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '78dcaaa8cee33438b91eca7f5c7f56a70fec9e80',
  'build_date': '2022-10-17T15:29:54.167373105Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

### Creating a new index

In [23]:
index_name = "lyrics_mood_classification"

es.indices.create(index=index_name,
                  mappings={
                      "dynamic": "strict",
                      "properties": {
                          "song_name":    {"type": "text"},
                          "artist_name":  {"type": "text"},
                          "lyrics":   {"type": "text"},
                          "mood":   {"type": "text"}
                      }
                  })

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'lyrics_mood_classification'}

In [24]:
es.indices.get(index=index_name)

{'lyrics_mood_classification': {'aliases': {},
  'mappings': {'dynamic': 'strict',
   'properties': {'artist_name': {'type': 'text'},
    'lyrics': {'type': 'text'},
    'mood': {'type': 'text'},
    'song_name': {'type': 'text'}}},
  'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'lyrics_mood_classification',
    'creation_date': '1671626551184',
    'number_of_replicas': '1',
    'uuid': 'ReVh0WfrQgeTlldhzum7tg',
    'version': {'created': '7170799'}}}}}

### Create a new document

In [25]:
import scrape_genius_lyrics as sgl

artist_name = "Eminem"
song_name = "Mockingbird"
lyrics = sgl.scrape_lyrics(artist_name, song_name)
mood = "sad"

Searching for "Eminem" by Mockingbird...
Done.
Song found.


In [26]:
ef.add_es_document(song_name, artist_name, lyrics, mood)

### Check if document was added

In [27]:
es.search(index=index_name)

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [31]:
ef.get_stored_lyrics_of_song(song_name, artist_name)

"TranslationsTürkçePortuguêsItalianoDeutschFrançaisEnglishMockingbird Lyrics[Intro]\nYeah\nI know sometimes\nThings may not always make sense to you right now\nBut ayy\nWhat'd Daddy always tell you?\nStraighten up, little soldier, stiffen up that upper lip\nWhat you cryin' about?\nYou got me\n\n[Verse 1]\nHailie, I know you miss your Mom, and I know you miss your Dad\nWhen I'm gone, but I'm tryna give you the life that I never had\nI can see you're sad, even when you smile, even when you laugh\nI can see it in your eyes, deep inside you wanna cry\n'Cause you're scared, I ain't there, Daddy's witchu in your prayers\nNo more cryin', wipe them tears, Daddy's here, no more nightmares\nWe gon' pull together through it, we gon' do it\nLainie—Uncle's crazy, ain't he? Yeah, but he loves you, girl, and you better know it\nWe're all we got in this world when it spins, when it swirls\nWhen it whirls, when it twirls, two little beautiful girls\nLookin' puzzled, in a daze, I know it's confusin' you

In [32]:
ef.get_stored_mood_of_song(song_name, artist_name)

'sad'

In [36]:
ef.get_stored_lyrics_of_song("bla", "bla") is None

True

In [34]:
ef.get_stored_mood_of_song("bla", "bla") is None